# Energy Meter Examples

## ARM Energy Probe

*NOTE*: `caiman` is required to collect data from the probe. Instructions on how to install it can be found here https://github.com/ARM-software/lisa/wiki/Energy-Meters-Requirements#arm-energy-probe-aep.

In [1]:
import logging
from conf import LisaLogging
LisaLogging.setup()

2016-12-08 12:40:07,000 INFO    : root         : Using LISA logging configuration:
2016-12-08 12:40:07,000 INFO    : root         :   /home/vagrant/lisa/logging.conf


#### Import required modules

In [2]:
# Generate plots inline
%matplotlib inline

import os

# Support to access the remote target
import devlib
from env import TestEnv

# RTApp configurator for generation of PERIODIC tasks
from wlgen import RTA, Ramp

## Target Configuration
The target configuration is used to describe and configure your test environment.
You can find more details in **examples/utils/testenv_example.ipynb**.

In [3]:
# Setup target configuration
my_conf = {

    # Target platform and board
    "platform"    : 'linux',
    "board"       : 'juno',
    "host"        : '192.168.0.1',

    # Folder where all the results will be collected
    "results_dir" : "EnergyMeter_AEP",

    # Define devlib modules to load
    "modules" : ["cpufreq"], # Required by rt-app calibration
    "exclude_modules" : [ 'hwmon' ],

    # Energy Meters Configuration for ARM Energy Probe
    "emeter" : {
        "instrument" : "aep",
        "conf" : {
            # Value of the shunt resistor in Ohm
            'resistor_values' : [0.099],
            # Device entry assigned to the probe on the host
            'device_entry'    : '/dev/ttyACM0',
        },
        'channel_map' : {
            'BAT' : 'BAT'
        }
    },
    
    # Tools required by the experiments
    "tools"   : [ 'trace-cmd', 'rt-app' ],
    
    # Comment this line to calibrate RTApp in your own platform
    # "rtapp-calib" :  {"0": 360, "1": 142, "2": 138, "3": 352, "4": 352, "5": 353},
}

In [4]:
# Initialize a test environment using:
te = TestEnv(my_conf, wipe=False, force_new=True)
target = te.target

03:58:29  INFO    :         Target - Using base path: /data/lisa
03:58:29  INFO    :         Target - Loading custom (inline) target configuration
03:58:29  INFO    :         Target - Devlib modules to load: ['bl', 'cpufreq']
03:58:29  INFO    :         Target - Connecting linux target:
03:58:29  INFO    :         Target -   username : root
03:58:29  INFO    :         Target -       host : 192.168.0.1
03:58:29  INFO    :         Target -   password : 
03:58:29  INFO    :         Target - Connection settings:
03:58:29  INFO    :         Target -    {'username': 'root', 'host': '192.168.0.1', 'password': ''}
03:58:33  INFO    :         Target - Initializing target workdir:
03:58:33  INFO    :         Target -    /root/devlib-target
03:58:39  INFO    :         Target - Topology:
03:58:39  INFO    :         Target -    [[0, 3, 4, 5], [1, 2]]
03:58:40  INFO    :       Platform - Loading default EM:
03:58:40  INFO    :       Platform -    /data/lisa/libs/utils/platforms/juno.json
03:58:40  W

## Workload Execution and Power Consumptions Samping

Detailed information on RTApp can be found in **examples/wlgen/rtapp_example.ipynb**.

Each **EnergyMeter** derived class has two main methods: **reset** and **report**.
 - The **reset** method will reset the energy meter and start sampling from channels specified in the target configuration. <br>
 - The **report** method will stop capture and will retrieve the energy consumption data. This returns an EnergyReport composed of the measured channels energy and the report file. Each of the samples can also be obtained, as you can see below.

In [5]:
# Create and RTApp RAMP task
rtapp = RTA(te.target, 'ramp', calibration=te.calibration())
rtapp.conf(kind='profile',
           params={
                'ramp' : Ramp(
                    start_pct =  60,
                    end_pct   =  20,
                    delta_pct =   5,
                    time_s    =   0.5).get()
          })

# EnergyMeter Start
te.emeter.reset()

rtapp.run(out_dir=te.res_dir)

# EnergyMeter Stop and samples collection
nrg_report = te.emeter.report(te.res_dir)

03:58:42  INFO    :          WlGen - Setup new workload ramp
03:58:42  INFO    :          RTApp - Workload duration defined by longest task
03:58:42  INFO    :          RTApp - Default policy: SCHED_OTHER
03:58:42  INFO    :          RTApp - ------------------------
03:58:42  INFO    :          RTApp - task [ramp], sched: using default policy
03:58:42  INFO    :          RTApp -  | calibration CPU: 1
03:58:42  INFO    :          RTApp -  | loops count: 1
03:58:42  INFO    :          RTApp - + phase_000001: duration 0.500000 [s] (5 loops)
03:58:42  INFO    :          RTApp - |  period   100000 [us], duty_cycle  60 %
03:58:42  INFO    :          RTApp - |  run_time  60000 [us], sleep_time  40000 [us]
03:58:42  INFO    :          RTApp - + phase_000002: duration 0.500000 [s] (5 loops)
03:58:42  INFO    :          RTApp - |  period   100000 [us], duty_cycle  55 %
03:58:42  INFO    :          RTApp - |  run_time  55000 [us], sleep_time  45000 [us]
03:58:42  INFO    :          RTApp - + phas

In [6]:
logging.info("Collected data:")
!tree $te.res_dir

03:58:50  INFO    : Collected data:


/data/lisa/results/EnergyMeter_AEP
├── energy.json
├── output.log
├── ramp_00.json
├── rt-app-ramp-0.log
└── samples.csv

0 directories, 5 files


## Power Measurements Data

In [7]:
logging.info("Measured channels energy:")
logging.info("%s", nrg_report.channels)

03:58:50  INFO    : Measured channels energy:
03:58:50  INFO    : {'BAT': 0.02506927080038548}


In [8]:
logging.info("Generated energy file:")
logging.info("  %s", nrg_report.report_file)
!cat $nrg_report.report_file

03:58:51  INFO    : Generated energy file:
03:58:51  INFO    :   /data/lisa/results/EnergyMeter_AEP/energy.json


{
    "BAT": 0.02506927080038548
}

In [9]:
logging.info("Samples collected for the BAT channel (only first 10)")
samples_file = os.path.join(te.res_dir, 'samples.csv')
!head $samples_file

03:58:52  INFO    : Samples collected for the BAT channel (only first 10)
